In [ ]:
%cd "C:\\Users\\Administrator\\Desktop\\强化学习\\中证50指数增强\\"
path = "C:\\Users\\Administrator\\Desktop\\强化学习\\中证50指数增强\\"

import warnings
warnings.filterwarnings("ignore")
import time
import numpy as np
import pandas as pd
#显示Dateframe所有列(参数设置为None代表显示所有行，也可以自行设置数字)
pd.set_option('display.max_columns',None)
#显示Dateframe所有行
pd.set_option('display.max_rows',None)
#设置Dataframe数据的显示长度，默认为50
pd.set_option('max_colwidth',None)
#不显示科学计数法，保留5位小数
pd.set_option('display.float_format', lambda x: '%.5f' % x)

from talib_factors import *
import datetime
import os
from tqdm import tqdm
import talib

## Factor Calculation Function

In [ ]:
train_df = pd.read_csv('数据//train_df.csv').drop(columns='Unnamed: 0')
columns_to_keep = get_factors(train_df).columns.to_list()

columns_to_keep

In [ ]:
def filter_extreme_3sigma(data, n=3, times=3): 
    # times进行times次3 sigma处理
    series = data.copy()
    for i in range(times):
        mean = series.mean()
        std = series.std()
        max_range = mean + n*std
        min_range = mean - n*std
        series = np.clip(series,min_range,max_range)
    return series

def job_get_factors(i, df, columns_to_keep, std_rolling, factor_background, window_size):
    tmp_df = df.iloc[i-(factor_background+window_size+std_rolling):i]
    tmp_df_2 = get_factors(train_df)(tmp_df)
    tmp_df_2 = tmp_df_2[columns_to_keep]
    tmp_df_2 = tmp_df_2.replace([np.inf, -np.inf], np.nan).fillna(0).reset_index(drop=True)
    columns = tmp_df_2.drop(columns=['date', 'tic', 'underlaying_symbols']).columns.to_list()
    #Standarlization
    tmp_df_3 = pd.DataFrame()
    for a in columns:
        tmp_df_2[a] = tmp_df_2[a].astype('float64')
        tmp_df_2[a] = filter_extreme_3sigma(tmp_df_2[a])
        tmp_df_2[a] = (tmp_df_2[a]-tmp_df_2[a].rolling(window=std_rolling, center=False).mean())/\
                       tmp_df_2[a].rolling(window=std_rolling, center=False).std()

    tmp_df_3 = tmp_df_3.append(tmp_df_2.drop(tmp_df_2.head(std_rolling).index))
    tmp_df_3 = tmp_df_3.replace([np.inf, -np.inf], np.nan).fillna(0)
    return tmp_df_3.drop(columns=['date', 'tic', 'underlaying_symbols']).tail(window_size).to_numpy()   

## Prepare training data

In [ ]:
std_period = 160
factor_background = 125
window_size = 85

In [ ]:
train_df = pd.read_csv('数据//train_df.csv').drop(columns='Unnamed: 0')
stocks = train_df.code.unique()
train_len = train_df[train_df.code == stocks[0]].shape[0]
train_obs = []
train_price = []
train_symbols = []
for a in tqdm(range(train_len-window_size-factor_background-std_period)):
    tmp_obs_df = pd.DataFrame()
    tmp_price = []
    for b in range(len(stocks)):
        tmp_df = train_df[train_df.code == stocks[b]].iloc[a:a+window_size+factor_background+std_period]
        tmp_df_1 = get_factors(tmp_df, window_size, std_period, factor_background, window_size)
        tmp_obs_df = tmp_obs_df.append(tmp_df_1)
        tmp_price.append(tmp_df.close.iloc[-1])
        if b == len(stocks)-1:
            tmp_price.append(tmp_df.date.iloc[-1])
        else:
            pass
    train_obs.append(tmp_obs_df.to_numpy())
    train_price.append(tmp_price)
    train_symbols.append(stocks)
train_obs = np.asarray(train_obs)
np.save('处理后数据//train_obs'+str(std_period)+'.npy', train_obs)
train_price = np.asarray(train_price)
np.save('处理后数据//train_price'+str(std_period)+'.npy', train_price)
train_symbols = np.asarray(train_symbols)
np.save('处理后数据//train_symbols'+str(std_period)+'.npy', train_symbols)

train_obs.shape, train_price.shape, train_symbols.shape

## Prepare testing data

In [ ]:
test_df = pd.read_csv('数据//test_df.csv').drop(columns='Unnamed: 0')
stocks = test_df.code.unique()
test_len = test_df[test_df.code == stocks[0]].shape[0]
test_obs = []
test_price = []
test_symbols = []
for a in tqdm(range(test_len-window_size-factor_background-std_period)):
    tmp_obs_df = pd.DataFrame()
    tmp_price = []
    for b in range(len(stocks)):
        tmp_df = test_df[test_df.code == stocks[b]].iloc[a:a+window_size+factor_background+std_period]
        tmp_df_1 = get_factors(tmp_df, window_size, std_period)
        tmp_obs_df = tmp_obs_df.append(tmp_df_1)
        tmp_price.append(tmp_df.close.iloc[-1])
        if b == len(stocks)-1:
            tmp_price.append(tmp_df.date.iloc[-1])
        else:
            pass
    test_obs.append(tmp_obs_df.to_numpy())
    test_price.append(tmp_price)
    test_symbols.append(stocks)
test_obs = np.asarray(test_obs)
np.save('处理后数据//test_obs'+str(std_period)+'.npy', test_obs)
test_price = np.asarray(test_price)
np.save('处理后数据//test_price'+str(std_period)+'.npy', test_price)
test_symbols = np.asarray(test_symbols)
np.save('处理后数据//test_symbols'+str(std_period)+'.npy', test_symbols)

test_obs.shape, test_price.shape, test_symbols.shape